In [1]:
#Python version: 3.10.7
import numpy as np
from scipy.stats import ttest_ind,ttest_rel
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from itertools import combinations
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import kruskal
import os

In [6]:
filepath = './data/database.csv'
df = pd.read_csv(filepath)
df.head()

,study_id,land_use_system,land_use_type,area_id,sample_set,sample_pooling,sample_plot_size_m,design_type,continent,country,...,OTU_number_absolute,OTU_number_relative,ACE,Chao1,pielou_evenness,berger_dominance,simpson,dominant_phylum,dominant_func_guild,notes
0,1,primary forest,natural,1,1,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,primary forest,natural,1,2,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,primary forest,natural,1,3,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,agricultural meadow,moderate,2,1,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,agricultural meadow,moderate,2,2,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.shape

(142, 35)

In [8]:
def getSoilDepthClass(val:str):
    #classes: 0-20 (allowed 0-25), 20-40 and 1-30 for bigger ranges
    if pd.isna(val):
        return "nan"

    val = str(val).strip()

    if val == "" or val.lower() == "nan":
        return "nan"

    # expect: "start-end"
    if "-" not in val:
        raise ValueError(f'Unmatching string with <start>-<end>.')
    start = int(val.split('-')[0])
    end = int(val.split('-')[1])

    if start >= 0 and end <= 25:
        return '0-20'
    elif start >= 20:
        return  '20-40'
    else:
        return '10-30'


In [9]:
df['soil_depth_cm'] = df['soil_depth_cm'].astype("string")
df['soilDepthClass'] = [getSoilDepthClass(x) for x in df['soil_depth_cm']]
df['soilDepthClass'].value_counts()

soilDepthClass
0-20     115
10-30     14
20-40     13
Name: count, dtype: int64

In [10]:
BASE_DIR = './img/'
os.makedirs(BASE_DIR,exist_ok=True)

In [11]:
df.columns

Index(['study_id', 'land_use_system', 'land_use_type', 'area_id', 'sample_set',
       'sample_pooling', 'sample_plot_size_m', 'design_type', 'continent',
       'country', 'country_region', 'climate_zone', 'precipitation_condition',
       'sampling_year', 'sampling_season', 'sampling_month',
       'soil_sample_number', 'soil_depth_cm', 'soil_type',
       'species_delimination_method', 'shannon_index',
       'total_number_of_species_in_study', 'species_richness_absolute',
       'species_richness_relative', 'OTU_total_in_study',
       'OTU_number_absolute', 'OTU_number_relative', 'ACE', 'Chao1',
       'pielou_evenness', 'berger_dominance', 'simpson', 'dominant_phylum',
       'dominant_func_guild', 'notes', 'soilDepthClass'],
      dtype='object')

In [12]:
#number of studies
n_studies = len(df['study_id'].unique())
print(f'Number of studies: {n_studies}')

Number of studies: 22


In [13]:
# distribution of land use systems
df['land_use_system'].value_counts()

land_use_system
monoculture                 26
primary forest              21
grassland                   21
arable land                 16
secondary forest            16
agricultural meadow         11
managed forest              11
former argicultural land    10
wetland                      4
clear cut                    3
wasteland                    3
Name: count, dtype: int64

In [14]:
#distribution of land use types
df['land_use_type'].value_counts()

land_use_type
strong anthropogene    52
moderate               51
natural                39
Name: count, dtype: int64

In [15]:
# design type
df['design_type'].value_counts()

design_type
field study    139
simulation       3
Name: count, dtype: int64

In [16]:
# continents
df['continent'].value_counts()

continent
europe           56
asia             53
south america    21
north america     7
australia         5
Name: count, dtype: int64

In [17]:
#country
df['country'].value_counts()

country
China            40
Latvia           30
Italy            16
Brazil           15
Indonesia         8
USA               7
Chile             6
Thailand          5
New Zealand       5
Switzerland       4
Netherlands       3
United Kindom     3
Name: count, dtype: int64

In [18]:
# region
df['country_region'].value_counts()

country_region
Jelgava district                            15
Berchidda                                   13
Guizhou Province                            10
Cesis distric                                9
Jambi Province                               8
Pernambuco                                   6
Valka district                               6
Songnen Plain                                6
Rondônia                                     6
Sanjiang Plain                               6
Puyehue National Park                        6
Xishuangbanna                                5
Lanzhou City                                 5
Middle Rio Grande Bosque                     5
Tha Kum-Huai Raeng Forest Reserve            5
Taojia River Basin                           4
Basel and surrounding area                   4
Henan Province                               4
Benjamin Constant, Amazonas                  3
Veluwe                                       3
Yorkshire Dales                              

In [19]:
# climate zone
df['climate_zone'].value_counts()

climate_zone
temperate      73
tropical       35
subtropical    34
Name: count, dtype: int64

In [20]:
# sampling year
startSampling = int(df['sampling_year'].min())
endSampling = int(df['sampling_year'].max())

print(f'Sampling: {startSampling}-{endSampling}')
df['sampling_year'].value_counts()

Sampling: 2004-2022


sampling_year
2007    40
2011    12
2020    10
2019    10
2012    10
2010     9
2017     9
2016     8
2021     8
2013     6
2008     5
2015     5
2022     4
2009     3
2004     3
Name: count, dtype: int64

In [21]:
# sampling season
df['sampling_season'].value_counts()

sampling_season
autumn    41
rainy     38
summer    34
dry       15
spring    14
Name: count, dtype: int64

In [22]:
# species delimination methods
df['species_delimination_method'].value_counts()

species_delimination_method
genetically                   107
morphological +genetically     30
morphological                   5
Name: count, dtype: int64

In [23]:
# dominant phylum
df['dominant_phylum'].value_counts()

dominant_phylum
Ascomycota       96
Basidiomycota     8
Mucoromycota      3
Basidomycota      3
Zygomycota        2
Name: count, dtype: int64

In [24]:
# dominant function
df['dominant_func_guild'].value_counts()

dominant_func_guild
saprophytic        73
ectomycorrhiza     10
pathogene           3
plant pathogene     2
Name: count, dtype: int64

# Plot occurences

In [25]:
# pie charts
def pieChart(labels:list,values:list,min_percent_inside:int=5, title="", doughnutNum:int=0,fontsize:int=25,colorDict: dict | None = None):
    total = sum(values)
    percents = [v / total * 100 for v in values]

    text_positions = []
    for p in percents:
        if p < min_percent_inside:
            text_positions.append("outside")
        else:
            text_positions.append("inside")

    texts = [str(v) for v in values]

    if colorDict is not None:
        colors = [
            colorDict.get(label, {}).get("fill", "#CCCCCC")
            for label in labels
        ]

        line_colors = [
            colorDict.get(label, {}).get("line", "#666666")
            for label in labels
        ]
    else:
        colors = None
        line_colors = None


    fig = go.Figure(
        go.Pie(
            labels=labels,
            values=values,

            text=texts,
            textinfo="text+percent",
            textposition=text_positions,

            insidetextorientation="radial",
            outsidetextfont=dict(size=12),
            marker=dict(colors=colors,line=dict(
                color=line_colors,
                width=1
            )) if colors else None,
            hole=doughnutNum
        )
    )

    fig.update_layout( 
        template = 'ggplot2',
        title={
                'text': f'<b> {title}</b>',
                'x':0.5,
                'y':0.96, 
                'xanchor':'center',
                'yanchor':'top'},
        font=dict(size=fontsize),
        height=800,width=1200
    )

    return fig

In [26]:
opacity = 0.7

landUseColorDict = {
    "monoculture": {
        "fill": f"rgba(139, 0, 0, {opacity})",
        "line": "rgb(139, 0, 0)",
    },
    "arable land": {
        "fill": f"rgba(178, 34, 34, {opacity})",
        "line": "rgb(178, 34, 34)",
    },
    "clear cut": {
        "fill": f"rgba(205, 92, 92, {opacity})",
        "line": "rgb(205, 92, 92)",
    },

    "primary forest": {
        "fill": f"rgba(46, 139, 87, {opacity})",
        "line": "rgb(46, 139, 87)",
    },
    "grassland": {
        "fill": f"rgba(102, 194, 133, {opacity})",
        "line": "rgb(102, 194, 133)",
    },
    "wetland": {
        "fill": f"rgba(60, 179, 113, {opacity})",
        "line": "rgb(60, 179, 113)",
    },

    "agricultural meadow": {
        "fill": f"rgba(230, 159, 0, {opacity})",
        "line": "rgb(230, 159, 0)",
    },
    "former argicultural land": {
        "fill": f"rgba(208, 140, 63, {opacity})",
        "line": "rgb(208, 140, 63)",
    },
    "secondary forest": {
        "fill": f"rgba(154, 205, 50, {opacity})",
        "line": "rgb(154, 205, 50)",
    },

    "managed forest": {
        "fill": f"rgba(106, 142, 174, {opacity})",
        "line": "rgb(106, 142, 174)",
    },
    "wasteland": {
        "fill": f"rgba(138, 138, 138, {opacity})",
        "line": "rgb(138, 138, 138)",
    },
}


In [27]:
BASE_DIR_PIE = os.path.join(BASE_DIR,'pie/')
os.makedirs(BASE_DIR_PIE,exist_ok=True)
counts = df["land_use_system"].value_counts()
labels = counts.index.tolist()
values = counts.values.tolist()

fig = pieChart(labels=labels,values=values,title='Occurences of land use systems in database',colorDict=landUseColorDict)
fig.write_image(os.path.join(BASE_DIR_PIE,'pie_land_use_system.png'))

In [28]:
# plot land use types
landUseTypesDict = {
    "moderate": {
        "fill": f"rgba(70, 130, 180, {opacity})",      # steelblue
        "line": "rgb(70, 130, 180)",
    },
    "strong anthropogene": {
        "fill": f"rgba(139, 0, 0, {opacity})",         # darkred
        "line": "rgb(139, 0, 0)",
    },
    "natural": {
        "fill": f"rgba(85, 107, 47, {opacity})",       # darkolivegreen
        "line": "rgb(85, 107, 47)",
    },
}

counts = df['land_use_type'].value_counts()
labels = counts.index.tolist()
values = counts.values.tolist()

fig = pieChart(labels=labels,values=values,title='Occurences of land use types in database',colorDict=landUseTypesDict)
fig.write_image(os.path.join(BASE_DIR_PIE,'pie_land_use_type.png'))

In [29]:
opacity = 0.7
continentColorDict = {
    "europe": {
        "fill": f"rgba(85, 107, 47, {opacity})",     # darkolivegreen
        "line": "rgb(85, 107, 47)",
    },
    "asia": {
        "fill": f"rgba(218, 165, 32, {opacity})",    # goldenrod
        "line": "rgb(218, 165, 32)",
    },
    "south america": {
        "fill": f"rgba(139, 0, 0, {opacity})",       # darkred
        "line": "rgb(139, 0, 0)",
    },
    "north america": {
        "fill": f"rgba(70, 130, 180, {opacity})",    # steelblue
        "line": "rgb(70, 130, 180)",
    },
    "australia": {
        "fill": f"rgba(255, 140, 0, {opacity})",     # darkorange
        "line": "rgb(255, 140, 0)",
    },
}

In [30]:
samplingSeasonColorDict = {
    "dry": {
        "fill": f"rgba(255, 140, 0, {opacity})",     # darkorange
        "line": "rgb(255, 140, 0)",
    },
    "summer": {
        "fill": f"rgba(139, 0, 0, {opacity})",       # darkred
        "line": "rgb(139, 0, 0)",
    },
    "rainy": {
        "fill": f"rgba(70, 130, 180, {opacity})",    # steelblue
        "line": "rgb(70, 130, 180)",
    },
    "autumn": {
        "fill": f"rgba(218, 165, 32, {opacity})",    # goldenrod
        "line": "rgb(218, 165, 32)",
    },
    "spring": {
        "fill": f"rgba(85, 107, 47, {opacity})",     # darkolivegreen
        "line": "rgb(85, 107, 47)",
    },
}

In [31]:
climateColorDict = {
    "temperate": {
        "fill": f"rgba(85, 107, 47, {opacity})",     # darkolivegreen
        "line": "rgb(85, 107, 47)",
    },
    "tropical": {
        "fill": f"rgba(139, 0, 139, {opacity})",     # darkmagenta
        "line": "rgb(139, 0, 139)",
    },
    "subtropical": {
        "fill": f"rgba(255, 140, 0, {opacity})",     # darkorange
        "line": "rgb(255, 140, 0)",
    },
}

In [32]:
# plot pie for rest 
titleDict = {
    'design_type': 'Design types',
    'continent': 'Continents',
    'country': 'Countries',
    'country_region': 'Country regions',
    'sampling_year': 'Sampling years',
    'sampling_season': 'Sampling seasons',
    'climate_zone': 'Climate zones',
    'species_delimination_method': 'Methods for species delimination',
    'dominant_func_guild': 'Dominant functional guilds',
    'dominant_phylum': 'Dominant phyla'
}
cols = ['design_type','continent','country','country_region','sampling_year','sampling_season','climate_zone','species_delimination_method','dominant_func_guild','dominant_phylum']
colorDictList = [None,continentColorDict,None,None,None,samplingSeasonColorDict,climateColorDict,None,None,None]
for idx, col in enumerate(cols):
    counts = df[col].value_counts()
    labels = counts.index.tolist()
    values = counts.values.tolist()
    color = colorDictList[idx]

    fig = pieChart(labels=labels,values=values,title=f'Number of {titleDict[col]} in database',colorDict=color)
    fig.write_image(os.path.join(BASE_DIR_PIE,f'pie_{col}.png'))

# Forest Plots

In [33]:
def forestChart(
    df: pd.DataFrame,xCol: str,yCol: str,xAxisTitle:str,yAxisTitle,
    legendTitle:str='',fileoutName: str | None = None,groupCol: str | None = None,title: str = "",
    *,
    colorDict: dict | None = None, point_size: int = 12,
    reverse_y: bool = False,side_by_side: bool = True,    offset_strength: float = 0.18,show_raw_points: bool = False,raw_point_opacity: float = 0.25,
    show_n: bool = True,n_prefix: str = "n=",n_y_offset: float | None = None,n_fontsize: int = 11,
):

    dff = df.copy()
    # group handling
    if groupCol is None:
        dff["_group"] = "all"
        groupCol_use = "_group"
    else:
        groupCol_use = groupCol

    # y 
    dff["_y"] = pd.to_numeric(dff[yCol], errors="coerce")
    dff = dff.dropna(subset=["_y", xCol, groupCol_use])

    agg = (
        dff.groupby([xCol, groupCol_use], sort=False)["_y"]
        .agg(y_mean="mean", y_low="min", y_high="max", n="count")
        .reset_index()
    )
    if n_y_offset is None:
        y_span = agg["y_high"].max() - agg["y_low"].min()
        n_y_offset = 0.04 * y_span if pd.notna(y_span) and y_span != 0 else 0.1

    x_levels = list(pd.Categorical(dff[xCol]).categories) if pd.api.types.is_categorical_dtype(dff[xCol]) else list(dff[xCol].dropna().unique())
    if not x_levels:
        x_levels = list(agg[xCol].unique())

    groups = list(agg[groupCol_use].unique())

    x_to_num = {x: i for i, x in enumerate(x_levels)}
    base_x = agg[xCol].map(x_to_num).astype(float)

    if side_by_side and len(groups) > 1:
        offs = np.linspace(-offset_strength, offset_strength, len(groups))
        off_map = {g: offs[i] for i, g in enumerate(groups)}

    fig = go.Figure()
    for g in groups:
        d = agg[agg[groupCol_use] == g].copy()
        marker_kwargs = dict(size=point_size)
        if colorDict is not None and g in colorDict:
            c = colorDict[g]
            if isinstance(c, dict):
                marker_kwargs["color"] = c.get("fill", None)
                marker_kwargs["line"] = dict(color=c.get("line", None), width=1.5)
            elif isinstance(c, str):
                marker_kwargs["color"] = c

        y_mean = d["y_mean"]
        y_low  = d["y_low"]
        y_high = d["y_high"]
        d["_xnum"] = d[xCol].map(x_to_num).astype(float)
        if side_by_side and len(groups) > 1:
            d["_xnum"] = d["_xnum"] + (np.linspace(-offset_strength, offset_strength, len(groups))[groups.index(g)])

        fig.add_trace(
            go.Scatter(
                x=d["_xnum"],
                y=y_mean,
                mode="markers",
                name=str(g),
                marker=marker_kwargs,
                opacity=0.7,
                error_y=dict(
                    type="data",
                    symmetric=False,
                    array=(y_high - y_mean),
                    arrayminus=(y_mean - y_low),
                    thickness=1.5,
                    width=0,
                ),
                customdata=np.stack([y_low, y_high, d["n"]], axis=1),
                hovertemplate=(
                    f"{groupCol_use}=%{{fullData.name}}<br>"
                    f"{xCol}=%{{x}}<br>"
                    f"mean=%{{y:.3f}}<br>"
                    f"min=%{{customdata[0]:.3f}}<br>"
                    f"max=%{{customdata[1]:.3f}}<br>"
                    f"n=%{{customdata[2]}}<extra></extra>"
                ),
            )
        )
        if show_n:
            fig.add_trace(
                go.Scatter(
                    x=d["_xnum"],
                    y=d["y_low"] - n_y_offset,
                    mode="text",
                    text=[f"{n_prefix}{int(v)}" for v in d["n"]],
                    textposition="middle center",
                    showlegend=False,
                    hoverinfo="skip",
                    textfont=dict(size=n_fontsize),
                )
            )

    if show_raw_points:
        dff["_xnum"] = dff[xCol].map(x_to_num).astype(float)

        if side_by_side and len(groups) > 1:
            offs = np.linspace(-offset_strength, offset_strength, len(groups))
            off_map = {g: offs[i] for i, g in enumerate(groups)}
            dff["_xnum"] = dff["_xnum"] + dff[groupCol_use].map(off_map).astype(float)

        jitter = (np.random.rand(len(dff)) - 0.5) * 0.06
        fig.add_trace(
            go.Scatter(
                x=dff["_xnum"] + jitter,
                y=dff["_y"],
                mode="markers",
                name="raw",
                marker=dict(size=6, opacity=raw_point_opacity, color="gray"),
                showlegend=False,
                hoverinfo="skip",
            )
        )

    fig.update_layout(
        legend=dict(
                x= 0.5,
                y= 1.14,
                xanchor='center',
                orientation = 'h',
                title_font_family="Times New Roman",
                font=dict(
                        family="Courier",
                        size=20,
                        color="black"
                ),
                title_text=legendTitle,
                bgcolor="white",
                bordercolor="lightgrey",
                borderwidth=2
        ),
        template = 'ggplot2',
        title={
                'text': f'<b> {title}</b>',
                'x':0.5,
                'y':0.98, 
                'xanchor':'center',
                'yanchor':'top'},
        font=dict(size=25),
        height=600,width=1300
        
    )

    fig.update_xaxes(
        tickmode="array",
        tickvals=list(range(len(x_levels))),
        ticktext=x_levels,
        title=xAxisTitle
    )
    fig.update_yaxes(title=yAxisTitle,range=[agg["y_low"].min() * 0.9 -n_y_offset, agg["y_high"].max()*1.05])


    if reverse_y:
        fig.update_yaxes(autorange="reversed")

    if fileoutName:
        fig.write_html(f"{fileoutName}.html")
        fig.write_image(f"{fileoutName}.png")

    return fig


In [34]:
metricNamesDict = {
    'shannon_index': 'Shannon index',
    'OTU_number_relative': 'Number of OTUs (relative)', 
    'Chao1': 'Chao1',
    'simpson': 'Simpson'
}

In [35]:
seasonColorDict = {
    'dry': 'darkorange',
    'summer': 'darkred',
    'rainy': 'steelblue',
    'autumn': 'goldenrod',
    'spring': 'darkolivegreen'
}
climateColorDict = {
    'temperate': 'darkolivegreen', 
    'tropical': 'darkmagenta', 
    'subtropical':'darkorange'
}
continentColorDict = {
    'europe': 'darkolivegreen', 
    'asia': 'goldenrod', 
    'south america': 'darkred', 
    'north america': 'steelblue', 
    'australia': 'darkorange'
}

soilSampleColors = {
    '0-20': 'steelblue',
    '10-30': 'darkred',
    '20-40': 'darkolivegreen'
}

facetColorList = [seasonColorDict,climateColorDict,continentColorDict,soilSampleColors]
facetTitels = ['Sampling seasons','climate zones','continents','soil depths']


In [36]:
df['land_use_system'].unique()

array(['primary forest', 'agricultural meadow', 'arable land',
       'former argicultural land', 'secondary forest', 'grassland',
       'monoculture', 'managed forest', 'clear cut', 'wetland',
       'wasteland'], dtype=object)

In [37]:
#per land use type
for idx, facet in enumerate(['sampling_season','climate_zone','continent','soilDepthClass']):
    colorDict = facetColorList[idx]
    for col in metricNamesDict.keys():
        out_Dir = os.path.join(BASE_DIR,f'{col}/','forest/')
        os.makedirs(out_Dir,exist_ok=True)
        fig = forestChart(
            df=df,
            xCol="land_use_type",
            yCol=col,
            groupCol=facet,
            title=f"{metricNamesDict[col]} based on land use type and {facetTitels[idx]}",
            show_n=True,
            n_prefix="n=",
            xAxisTitle='Land use intensity',
            legendTitle=facetTitels[idx],
            yAxisTitle=metricNamesDict[col],
            colorDict=colorDict
        )
        filename = f'land_use_type_2_{col}_by_{facet}'
        fig.write_image(os.path.join(out_Dir,f'{filename}.png'))
        fig.write_html(os.path.join(out_Dir,f'{filename}.html'))


In [38]:
landUseSystemShorts = {
    'primary forest': 'PF', 
    'agricultural meadow': 'AM', 
    'arable land': 'AL',
    'former argicultural land': 'FAL', 
    'secondary forest': 'Sf', 
    'grassland': 'GL',
    'monoculture': 'MC', 
    'managed forest': 'MF', 
    'clear cut': 'CC', 
    'wetland': 'WET',
    'wasteland': 'WASTE'
} 

In [39]:
landUseTypesColorDict = {
    "moderate": 'steelblue',
    "strong anthropogene": 'darkred',
    "natural":'darkolivegreen',
}

In [40]:
df['land_use_system_short'] = [landUseSystemShorts[x] for x in df['land_use_system'].values]

In [41]:
#per land use type
facetColorList = [seasonColorDict,climateColorDict,continentColorDict,soilSampleColors,landUseTypesColorDict]
facetTitels = ['Sampling seasons','climate zones','continents','soil depths', 'Land use intensity']

for idx, facet in enumerate(['sampling_season','climate_zone','continent','soilDepthClass','land_use_type']):
    colorDict = facetColorList[idx]
    for col in metricNamesDict.keys():
        out_Dir = os.path.join(BASE_DIR,f'{col}/','forest/')
        os.makedirs(out_Dir,exist_ok=True)
        fig = forestChart(
            df=df,
            xCol="land_use_system_short",
            yCol=col,
            groupCol=facet,
            title=f"{metricNamesDict[col]} based on land use system {facetTitels[idx]}",
            show_n=True,
            n_prefix="n=",
            xAxisTitle='Land use system',
            legendTitle=facetTitels[idx],
            yAxisTitle=metricNamesDict[col],
            colorDict=colorDict
        )
        filename = f'land_use_system_2_{col}_by_{facet}'
        fig.write_image(os.path.join(out_Dir,f'{filename}.png'))
        fig.write_html(os.path.join(out_Dir,f'{filename}.html'))


# Plot mean values against conditions

In [42]:
# group by: study_id , area_id , land_use_system , land_use_type , country_region and soil_depth
group_cols = [
    "study_id",
    "area_id",
    "land_use_system",
    "land_use_type",
    "country_region",
    "soil_depth_cm",
]


In [43]:
landUseTypeColorDict = {
    'natural':'darkolivegreen',
    'moderate': 'steelblue',
    'strong anthropogene': 'darkred'
}

In [44]:
df_field_study = df.where(df['design_type'] == 'field study').dropna(how='all')
df_field_study.head()

,study_id,land_use_system,land_use_type,area_id,sample_set,sample_pooling,sample_plot_size_m,design_type,continent,country,...,ACE,Chao1,pielou_evenness,berger_dominance,simpson,dominant_phylum,dominant_func_guild,notes,soilDepthClass,land_use_system_short
0,1.0,primary forest,natural,1.0,1.0,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-20,PF
1,1.0,primary forest,natural,1.0,2.0,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-20,PF
2,1.0,primary forest,natural,1.0,3.0,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-20,PF
3,1.0,agricultural meadow,moderate,2.0,1.0,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-20,AM
4,1.0,agricultural meadow,moderate,2.0,2.0,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-20,AM


In [45]:
for metric in metricNamesDict.keys():
    out_Dir = os.path.join(BASE_DIR,f'{metric}/','precip_2_continent/')
    os.makedirs(out_Dir,exist_ok=True)
    if df_field_study[metric].dtype != 'float64':
        continue
    print(metric)
    df_agg = (
        df_field_study.groupby(["precipitation_condition", "land_use_type",'study_id'], dropna=False, sort=False)
        .agg(metric=(metric, "mean"))
        .reset_index()
        )
    df_agg = df_agg[~df_agg['metric'].isna()]
    fig = px.scatter(
        df_agg,
        x="precipitation_condition",
        y='metric',
        color="land_use_type",
        color_discrete_map=landUseTypeColorDict,
        trendline='ols',
        custom_data=['study_id']
    )
    fig.update_layout(
            margin=dict(t=140),
            legend=dict(
                    x= 0.5,
                    y= 1.12,
                    xanchor='center',
                    orientation = 'h',
                    title_font_family="Times New Roman",
                    font=dict(
                            family="Courier",
                            size=20,
                            color="black"
                    ),
                    title_text='Land use intensity',
                    bgcolor="white",
                    bordercolor="lightgrey",
                    borderwidth=2
            ),
            template = 'ggplot2',
            title={
                    'text': f'<b> Averaged {metricNamesDict[metric]} to precipitation <br>in sampling month by land use intensity</b>',
                    'x':0.5,
                    'y':0.96, 
                    'xanchor':'center',
                    'yanchor':'top'},
            font=dict(size=25),
            height=700,width=1300
            
        )
    fig.update_yaxes(title=f'\u0394 {metricNamesDict[metric]}')
    fig.update_xaxes(title='Precipitation [mm/month]')
    fig.update_traces(hovertemplate=(
        f"metric=%{{fullData.name}}<br>"
        f"precipitation_condition=%{{x}}<br>"
        f"studyID=%{{customdata[0]}}<br>"
    ))
    filename = f'precipitation_to_{metric}_all'
    fig.write_image(os.path.join(out_Dir,f'{filename}.png'))
    fig.write_html(os.path.join(out_Dir,f'{filename}.html'))

shannon_index
OTU_number_relative
Chao1
simpson


In [46]:
for metric in metricNamesDict.keys():
    if df_field_study[metric].dtype != 'float64':
        continue
    print(metric)
    out_Dir = os.path.join(BASE_DIR,f'{metric}/','precip_2_continent/')
    os.makedirs(out_Dir,exist_ok=True)
    for continent in df_field_study['continent'].unique():
        if continent == 'nan':
            continue
        subset = df_field_study[df_field_study["continent"] == continent].copy()
        if subset[metric].dropna().shape[0] < 1:
            continue
        df_agg = (
            subset.groupby(["precipitation_condition", "land_use_type","continent","study_id"], dropna=False, sort=False)
            .agg(metric=(metric, "mean"))
            .reset_index()
            )
        df_agg = df_agg[~df_agg['metric'].isna()]
        fig = px.scatter(
            df_agg,
            x="precipitation_condition",
            y='metric',
            color="land_use_type",
            color_discrete_map=landUseTypeColorDict,
            trendline='ols',
            custom_data=['study_id']
        )
        fig.update_layout(
                margin=dict(t=160),
                legend=dict(
                        x= 0.5,
                        y= 1.17,
                        xanchor='center',
                        orientation = 'h',
                        title_font_family="Times New Roman",
                        font=dict(
                                family="Courier",
                                size=20,
                                color="black"
                        ),
                        title_text='Land use intensity',
                        bgcolor="white",
                        bordercolor="lightgrey",
                        borderwidth=2
                ),
                template = 'ggplot2',
                title={
                        'text': f'<b> Averaged {metricNamesDict[metric]} to precipitation <br>in sampling month by land use intensity in {continent}</b>',
                        'x':0.5,
                        'y':0.96, 
                        'xanchor':'center',
                        'yanchor':'top'},
                font=dict(size=25),
                height=700,width=1400
                
            )
        
        fig.update_yaxes(title=f'\u0394 {metricNamesDict[metric]}')
        fig.update_xaxes(title='Precipitation [mm/month]')
        fig.update_traces(hovertemplate=(
                    f"metric=%{{fullData.name}}<br>"
                    f"precipitation_condition=%{{x}}<br>"
                    f"studyID=%{{customdata[0]}}<br>"
                ))
        filename = f'precipitation_to_{metric}_{continent}'
        fig.write_image(os.path.join(out_Dir,f'{filename}.png'))
        fig.write_html(os.path.join(out_Dir,f'{filename}.html'))
        # fig.show()

shannon_index
OTU_number_relative
Chao1
simpson


In [47]:
df.columns

Index(['study_id', 'land_use_system', 'land_use_type', 'area_id', 'sample_set',
       'sample_pooling', 'sample_plot_size_m', 'design_type', 'continent',
       'country', 'country_region', 'climate_zone', 'precipitation_condition',
       'sampling_year', 'sampling_season', 'sampling_month',
       'soil_sample_number', 'soil_depth_cm', 'soil_type',
       'species_delimination_method', 'shannon_index',
       'total_number_of_species_in_study', 'species_richness_absolute',
       'species_richness_relative', 'OTU_total_in_study',
       'OTU_number_absolute', 'OTU_number_relative', 'ACE', 'Chao1',
       'pielou_evenness', 'berger_dominance', 'simpson', 'dominant_phylum',
       'dominant_func_guild', 'notes', 'soilDepthClass',
       'land_use_system_short'],
      dtype='object')

In [48]:
df['soil_depth_cm'].value_counts()

soil_depth_cm
0-20     28
1-10     23
0-10     19
1-20     19
10-30    11
30-40     8
0-15      6
20-40     5
0-25      5
1-15      5
1-5       4
1-1       3
10-10     3
1-30      3
Name: count, dtype: Int64

In [49]:
opacity = 0.7

soilDepthColorDict = {
    "0-20": {
        "fill": f"rgba(139, 0, 0, {opacity})",        # darkred
        "line": "rgb(139, 0, 0)",
    },
    "20-40": {
        "fill": f"rgba(85, 107, 47, {opacity})",      # darkolivegreen
        "line": "rgb(85, 107, 47)",
    },
    "10-30": {
        "fill": f"rgba(70, 130, 180, {opacity})",     # steelblue
        "line": "rgb(70, 130, 180)",
    },
}


In [50]:
counts = df['soilDepthClass'].value_counts()
labels = counts.index.tolist()
values = counts.values.tolist()

fig = pieChart(labels=labels,values=values,title='Occurences of soil depth [cm] from samplings in database',colorDict=soilDepthColorDict)
fig.write_image(os.path.join(BASE_DIR_PIE,f'pie_soil_DepthClass.png'))
fig.show()

In [51]:
out_Dir = os.path.join(BASE_DIR,'shannon_index/')
fig = px.violin(
    df,
    x="soilDepthClass",         
    y='shannon_index',
    color="land_use_type",       
    points="all",
    color_discrete_map=landUseTypeColorDict,
)

fig.update_layout(
    margin=dict(t=140),
    violinmode="group",
    violingap=0,
    violingroupgap=0,
    legend=dict(
        x=0.5,
        y=1.1,
        xanchor="center",
        yanchor="middle",
        orientation="h",
        title_text=f"<b>Land use intensity</b>",
        bgcolor="white",
        bordercolor="gray",
        borderwidth=2,
        font=dict(family="Courier", size=18, color="black"),
    ),
    template='ggplot2',
    title=dict(
        text=f"<b>Shannon index based on soil depth and land use intensity</b>",
        x=0.5, y=0.97,xanchor="center"
    ),
    height=700,
    width=1200,
    font=dict(size=25),
)

fig.update_xaxes(title_text='Soil depth [cm]',tickfont=dict(size=25))
fig.update_xaxes(
    categoryorder="array",
    categoryarray=["0-20", "10-30", "20-40"]
)
fig.update_yaxes(title_text='Shannon diversity',tickfont=dict(size=25))
fig.update_traces(meanline_visible=True,pointpos=0,jitter=0.8)
fig.update_traces(
    marker=dict(size=4, opacity=0.5),
)
filename = f'land_use_type_2_soil_depth_shannon_index'
fig.write_image(os.path.join(out_Dir,f'{filename}.png'))
fig.write_html(os.path.join(out_Dir,f'{filename}.html'))
fig.show()

In [52]:
val = 'shannon_index'
valName = 'Shannon index'
out_Dir = os.path.join(BASE_DIR, f'{val}/')

fig = px.box(
    df,
    x="soilDepthClass",
    y=val,
    color="land_use_type",
    points="all",  
    color_discrete_map=landUseTypeColorDict,
)

fig.update_layout(
    margin=dict(t=140),
    boxmode="group", 
    legend=dict(
        x=0.5,
        y=1.1,
        xanchor="center",
        yanchor="middle",
        orientation="h",
        title_text=f"<b>Land use intensity</b>",
        bgcolor="white",
        bordercolor="gray",
        borderwidth=2,
        font=dict(family="Courier", size=18, color="black"),
    ),
    template='ggplot2',
    title=dict(
        text=f"<b>{valName} based on soil depth and land use intensity</b>",
        x=0.5,
        y=0.97,
        xanchor="center"
    ),
    height=700,
    width=1200,
    font=dict(size=25),
)

fig.update_xaxes(
    title_text='Soil depth [cm]',
    tickfont=dict(size=25),
    categoryorder="array",
    categoryarray=["0-20", "10-30", "20-40"]
)

fig.update_yaxes(
    title_text=valName,
    tickfont=dict(size=25)
)

fig.update_traces(
    marker=dict(size=4, opacity=0.5),
    jitter=0.5,pointpos=0 
)

filename = f'land_use_type_2_soil_depth_{val}_box'

fig.write_image(os.path.join(out_Dir, f'{filename}.png'))
fig.write_html(os.path.join(out_Dir, f'{filename}.html'))

fig.show()


In [53]:
vals = ['shannon_index','OTU_number_relative','simpson']
valNames = ['Shannon index', 'Relative number of OTUs','Simpson']

for idx, val in enumerate(vals):
    out_Dir = os.path.join(BASE_DIR, f'{val}/')
    valName = valNames[idx]

    fig = px.box(
        df,
        x="land_use_type",
        y=val,
        color="land_use_type",
        points="all",
        color_discrete_map=landUseTypeColorDict,
    )

    fig.update_layout(
        margin=dict(t=140),
        boxmode="group", 
        legend=dict(
            x=0.5,
            y=1.1,
            xanchor="center",
            yanchor="middle",
            orientation="h",
            title_text=f"<b>Land use intensity</b>",
            bgcolor="white",
            bordercolor="gray",
            borderwidth=2,
            font=dict(family="Courier", size=18, color="black"),
        ),
        template='ggplot2',
        title=dict(
            text=f"<b>{valName} against land use intensity</b>",
            x=0.5,
            y=0.97,
            xanchor="center"
        ),
        height=700,
        width=1200,
        font=dict(size=25),
    )

    fig.update_xaxes(
        title_text='Land use intensity',
        tickfont=dict(size=25),
    )

    fig.update_yaxes(
        title_text=valName,
        tickfont=dict(size=25)
    )

    fig.update_traces(
        marker=dict(size=4, opacity=0.5),
        jitter=0.5,pointpos=0 
    )

    filename = f'land_use_type_2_{val}_box'

    fig.write_image(os.path.join(out_Dir, f'{filename}.png'))
    fig.write_html(os.path.join(out_Dir, f'{filename}.html'))


# Effect size: Comparing 2 groups against eachother
- Cohens d
- Hedges g
- p-Values
- t-Test

In [54]:
def getCohensD(x:list,y:list,pairwise:bool=False):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    x_std = np.std(x,ddof=1)
    y_std = np.std(y,ddof=1)

    if pairwise:
        std_pooled = np.sqrt(((x_std**2+y_std**2)/2))
    else: 
        n_x = len(x)
        n_y = len(y)
        numerator = ((n_x-1) * x_std**2) + ((n_y-1) * y_std**2) #oben: Zähler
        denominator = n_x + n_y -2 #unten: Nenner
        std_pooled = np.sqrt(numerator/denominator)

    cohensD = (x_mean-y_mean) / std_pooled
    return cohensD

In [55]:
def getStandardError(arr1:list,arr2:list,g:float):
    n1 = len(arr1)
    n2 = len(arr2)
    se = np.sqrt(((n1+n2)/(n1*n2))+(g**2/(2*(n1+n2))))
    return se

In [56]:
def getConfidenceInterval(arr1:list,arr2:list, g:float):
    se = getStandardError(arr1=arr1,arr2=arr2,g=g)
    minCI = g - 1.96 * se
    maxCI = g + 1.96 * se

    return minCI, maxCI

In [57]:
def getHedgesGEffectSize(x:list,y:list,pairwise:bool=False):
    n_x = len(x)
    n_y = len(y)

    d = getCohensD(x=x,y=y,pairwise=pairwise)

    g = d * (1-(3/(4*(n_x+n_y)-9)))
    return g

In [58]:
def getTTest(x:list,y:list,pairwise:bool=False):
    
    if pairwise:
        t_stat, p_value = ttest_rel(x, y, nan_policy="omit") #omit will over go nan and dont include them and wont throw errors
    else: 
        t_stat, p_value = ttest_ind(x, y, nan_policy="omit")
    
    return t_stat, p_value

In [59]:
#combine classes for tests
df['land_use_type_continent'] = (
    df['land_use_type'].astype(str) + '-' + df['continent'].astype(str)
)
df['land_use_type_sampling_season'] = (
    df['land_use_type'].astype(str) + '-' + df['sampling_season'].astype(str)
)

df['land_use_type_soilDepth'] = (
    df['land_use_type'].astype(str) + '-' + df['soilDepthClass'].astype(str)
)

df['land_use_type_climate_zone'] = (
    df['land_use_type'].astype(str) + '-' + df['climate_zone'].astype(str)
)


In [60]:
df.head()

,study_id,land_use_system,land_use_type,area_id,sample_set,sample_pooling,sample_plot_size_m,design_type,continent,country,...,simpson,dominant_phylum,dominant_func_guild,notes,soilDepthClass,land_use_system_short,land_use_type_continent,land_use_type_sampling_season,land_use_type_soilDepth,land_use_type_climate_zone
0,1,primary forest,natural,1,1,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,0-20,PF,natural-europe,natural-autumn,natural-0-20,natural-temperate
1,1,primary forest,natural,1,2,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,0-20,PF,natural-europe,natural-autumn,natural-0-20,natural-temperate
2,1,primary forest,natural,1,3,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,0-20,PF,natural-europe,natural-autumn,natural-0-20,natural-temperate
3,1,agricultural meadow,moderate,2,1,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,0-20,AM,moderate-europe,moderate-summer,moderate-0-20,moderate-temperate
4,1,agricultural meadow,moderate,2,2,False,NaN,field study,europe,Latvia,...,NaN,NaN,NaN,NaN,0-20,AM,moderate-europe,moderate-summer,moderate-0-20,moderate-temperate


In [61]:
col = 'shannon_index' # 'OTU_numer_relative','simpson'
pairCol = 'climate_zone' # 'land_use_type', 'continent', 'soilDepthClass'
#average multiple samples with same sampling condition in one area of a study to remove sampling number bias
df_grouped = (
    df
    .groupby(["study_id", "area_id",pairCol])[col]
    .mean()
    .reset_index()
)

landusetypes = df_grouped[pairCol].dropna().unique()
comparingPairs = list(combinations(landusetypes, 2))

for firstType, secondType in comparingPairs:
    print(f'First type: {firstType}, Second type: {secondType}')
    firstStudySubset = df_grouped.where(df_grouped[pairCol]==firstType).dropna(how='all')  
    secondStudySubset = df_grouped.where(df_grouped[pairCol]==secondType).dropna(how='all')
    x = firstStudySubset[col].dropna().values
    y = secondStudySubset[col].dropna().values

    if len(x) < 2 or len(y) < 2:
        continue

    #cohens d 
    d = getCohensD(x=x,y=y,pairwise=False)
    #Hedges g
    g = getHedgesGEffectSize(x=x,y=y,pairwise=False)
    #coeffiecnece intervall
    minCI,maxCI = getConfidenceInterval(arr1=x,arr2=y,g=g)
    #t test
    tTest, pValue = getTTest(x=x,y=y,pairwise=False)

    print(f'Cohens d: {d}\nHedges G: {g}\nCI: {minCI}  -  {maxCI}\nT Test: {tTest}\np-Wert:{pValue}\n')

First type: temperate, Second type: tropical
Cohens d: 0.4200867644241304
Hedges G: 0.4144353729744784
CI: -0.13224812249458706  -  0.9611188684435439
T Test: 1.520658864290889
p-Wert:0.1339721384724274

First type: temperate, Second type: subtropical
Cohens d: 0.5268774399285293
Hedges G: 0.519241535002029
CI: -0.07302945653534476  -  1.1115125265394026
T Test: 1.7679277507287583
p-Wert:0.08293963111080166

First type: tropical, Second type: subtropical
Cohens d: 0.0169225869319273
Hedges G: 0.016546529444551136
CI: -0.6408685662837551  -  0.6739616251728574
T Test: 0.050453406313252055
p-Wert:0.9600563233318902



# Anova tests: Compare multiple groups against eachother

### Anova simple: (Analysis of variance)
- Compares mean of multiple group and calculates if there really different or it happend by random
    - $H_0$ All groups have same average
    - $H_1$ At least one group is different
- It doesn't show, which group is different
- Condition:
    - independent observations
    - normal distrubited data
    - similiar variances
- Significance based on predefined $\alpha$ (mostly 0.05 or 0.01)
    - if $p<\alpha$ : significant
    - else not significant

In [62]:
#prepare dataframa and remove nan or empty strings
df_cleaned = df.copy()
#shannon strict to number
df_cleaned['shannon_index'] = pd.to_numeric(df_cleaned['shannon_index'],errors='coerce')
#clean groups
df_cleaned['land_use_type'] = df_cleaned['land_use_type'].astype("string")
df_cleaned.loc[df_cleaned["land_use_type"].isin(["","nan","None"]),"land_use_type"] = pd.NA

df_cleaned = df_cleaned.dropna(subset=["shannon_index",'land_use_type'])
df_cleaned = (
    df_cleaned
    .groupby(["study_id", "area_id",'land_use_type','continent','soilDepthClass','sampling_season','climate_zone'])
    .agg({
        "shannon_index": "mean",
        "simpson": "mean",
        "OTU_number_relative": "mean"
    })
    .reset_index()
)
counts = df_cleaned["land_use_type"].value_counts()
df_cleaned = df_cleaned[df_cleaned["land_use_type"].isin(counts[counts>=2].index)]
df_cleaned.head()

,study_id,area_id,land_use_type,continent,soilDepthClass,sampling_season,climate_zone,shannon_index,simpson,OTU_number_relative
0,1,1,natural,europe,0-20,autumn,temperate,2.600000,NaN,NaN
1,1,1,natural,europe,10-30,autumn,temperate,2.683333,NaN,NaN
2,1,1,natural,europe,20-40,autumn,temperate,2.416667,NaN,NaN
3,1,2,moderate,europe,0-20,summer,temperate,2.933333,NaN,NaN
4,1,2,moderate,europe,10-30,summer,temperate,2.250000,NaN,NaN


In [63]:
# one way analysis: look for differences in a single group
model = ols(
    "shannon_index ~ C(land_use_type)",
    data=df_cleaned
).fit()
anovaTable = sm.stats.anova_lm(model,type=2)
print(anovaTable)

                    df      sum_sq   mean_sq         F    PR(>F)
C(land_use_type)   2.0    5.824900  2.912450  1.136068  0.325795
Residual          87.0  223.035096  2.563622       NaN       NaN


In [64]:
# anova with multiple factors: i. ex. land usage and season
# tow way: looks which factor has which effent independently
model = ols(
    f"shannon_index ~ C(land_use_type) + C(sampling_season)",
    data=df_cleaned
).fit()

anova = sm.stats.anova_lm(model, type=2)
print(anova)

                      df      sum_sq   mean_sq         F    PR(>F)
C(land_use_type)     2.0    5.824900  2.912450  1.202292  0.305683
C(sampling_season)   4.0   21.974715  5.493679  2.267853  0.068735
Residual            83.0  201.060381  2.422414       NaN       NaN


In [65]:
# multiplae factor with interaction
model = ols(
    f"shannon_index ~ C(land_use_type) * C(sampling_season)",
    data=df_cleaned
).fit()

anova = sm.stats.anova_lm(model, type=2)
print(anova)

                                       df      sum_sq   mean_sq         F  \
C(land_use_type)                      2.0    5.824900  2.912450  1.108962   
C(sampling_season)                    4.0   21.974715  5.493679  2.091807   
C(land_use_type):C(sampling_season)   8.0    4.089098  0.511137  0.194624   
Residual                             75.0  196.971283  2.626284       NaN   

                                       PR(>F)  
C(land_use_type)                     0.335249  
C(sampling_season)                   0.090244  
C(land_use_type):C(sampling_season)  0.990883  
Residual                                  NaN  


In [66]:
# anova with multiple factors: i. ex. land usage and season
model = ols(
    f"simpson ~ C(land_use_type) + C(sampling_season) + C(continent) + C(soilDepthClass) + C(climate_zone)",
    data=df_cleaned
).fit()

anova = sm.stats.anova_lm(model, type=2)
print(anova)

                      df    sum_sq   mean_sq           F        PR(>F)
C(land_use_type)     2.0  0.041117  0.020559   10.313811  4.404028e-04
C(sampling_season)   4.0  7.039390  1.759848  882.876613  5.171856e-29
C(continent)         4.0  0.018688  0.004672    2.343817  7.913707e-02
C(soilDepthClass)    2.0  0.000418  0.000209    0.104954  9.007187e-01
C(climate_zone)      2.0  0.015329  0.007664    3.844997  3.346276e-02
Residual            28.0  0.055813  0.001993         NaN           NaN


In [67]:
df_test = df.groupby(
    ['sampling_season']

)['OTU_number_relative'].mean().reset_index()
df_test

,sampling_season,OTU_number_relative
0,autumn,0.185444
1,dry,0.672500
2,rainy,0.256333
3,spring,0.235929
4,summer,0.298474


# Post-hoc test
- if Anova shows significance: *Turkey-Test*
- Measures which groups are different

In [68]:
metric = 'shannon_index' # 'OTU_number_relative','simpson'

cols = ['land_use_type','continent','sampling_season','soilDepthClass','climate_zone']
for col in cols:
    print(col)
    #average multiple samples with same sampling condition in one area of a study to remove sampling number bias
    df_temp = df_cleaned[~df_cleaned[metric].isna()]
    df_temp = (
        df_temp
        .groupby(["study_id", "area_id",col])[metric]
        .mean()
        .reset_index()
    )
    if len(df_temp[col].unique()) < 2:
        continue
    turkey = pairwise_tukeyhsd(
        endog=df_temp[metric].dropna(),
        groups=df_temp[col].dropna(),
        alpha=0.05
    )
    print(turkey)

land_use_type
       Multiple Comparison of Means - Tukey HSD, FWER=0.05        
 group1         group2       meandiff p-adj   lower  upper  reject
------------------------------------------------------------------
moderate             natural   0.6162 0.4391 -0.5823 1.8146  False
moderate strong anthropogene   0.4664 0.5642 -0.6238 1.5567  False
 natural strong anthropogene  -0.1497 0.9489 -1.3109 1.0114  False
------------------------------------------------------------------
continent
       Multiple Comparison of Means - Tukey HSD, FWER=0.05       
    group1        group2    meandiff p-adj   lower  upper  reject
-----------------------------------------------------------------
         asia     australia  -0.8133 0.8181 -2.9304 1.3038  False
         asia        europe  -1.5076 0.0511 -3.0198 0.0047  False
         asia north america   0.1007 0.9999 -2.0164 2.2178  False
         asia south america  -0.9771 0.4604 -2.6183 0.6641  False
    australia        europe  -0.6942 0.9286 -